In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
titles_path = r"C:\Users\weism\OneDrive\life stuff\Personal Projects\imdb database analysis\title.basics.tsv"
names_path = r"C:\Users\weism\OneDrive\life stuff\Personal Projects\imdb database analysis\name.basics.tsv"
names_path = r"C:\Users\weism\OneDrive\life stuff\Personal Projects\imdb database analysis\name.basics.tsv"
principals_path = r"C:\Users\weism\OneDrive\life stuff\Personal Projects\imdb database analysis\title.principals.tsv"
ratings_path = r"C:\Users\weism\OneDrive\life stuff\Personal Projects\imdb database analysis\title.ratings.tsv"

In [3]:
titles_df = pd.read_csv(titles_path, delimiter="\t")

C:\Users\weism\AppData\Local\Temp\ipykernel_6188\3825395804.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  titles_df = pd.read_csv(titles_path, delimiter="\t")


In [4]:
names_df = pd.read_csv(names_path, delimiter="\t")

In [6]:
principals_df = pd.read_csv(principals_path, delimiter="\t")

In [5]:
ratings_df = pd.read_csv(ratings_path, delimiter="\t")

In [7]:
titles_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [8]:
names_df.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0072308,tt0053137,tt0050419"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0075213,tt0038355,tt0037382,tt0117057"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0057345,tt0056404,tt0054452,tt0049189"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0080455,tt0072562,tt0077975,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050976,tt0050986,tt0069467,tt0083922"


In [9]:
principals_df.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N


In [10]:
ratings_df.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1982
1,tt0000002,5.8,265
2,tt0000003,6.5,1839
3,tt0000004,5.5,178
4,tt0000005,6.2,2625


Note: titles_df and ratings_df appear to have a one to one relationship. This means that the two tables should be combined

### Replace \N with pd.Na

In [11]:
ratings_df = ratings_df.replace(r'^\\N$', pd.NA, regex=True)

In [12]:
names_df = names_df.replace(r'^\\N$', pd.NA, regex=True)

In [ ]:
principals_df = principals_df.replace(r'^\\N$', pd.NA, regex=True)

## titles_df Processing

In [70]:
titles_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,<NA>,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,<NA>,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,<NA>,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,<NA>,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,<NA>,1,"Comedy,Short"


In [545]:
# Verify uniqueness and ensure no nulls of "tconst" in titles_df
titles_df["tconst"].nunique() == titles_df.shape[0]

True

In [547]:
# Verify uniqueness  and ensure no nulls of "tconst" in ratings_df
ratings_df["tconst"].nunique() == ratings_df.shape[0]

True

In [549]:
# Check to see if all titles have a rating. It can be seen that not all titles have a rating, 
# meaning left join is necessary to combine.
print("count of titles: ", titles_df.shape[0])
print("count of ratings: ", ratings_df.shape[0])

count of titles:  9966186
count of ratings:  1324473


In [551]:
# Combine the two dataframes as "titles_ratings_df"
titles_ratings_df = pd.merge(titles_df, ratings_df, on='tconst', how='left')

In [1]:
# Ensure no lost titles or duplicates resulting from join and view result of join
print(titles_ratings_df.shape[0] == titles_df.shape[0])
titles_ratings_df.head()

NameError: name 'titles_ratings_df' is not defined

In [128]:
titles_ratings_df[["startYear", "endYear", "runtimeMinutes"]] \
= titles_ratings_df[["startYear", "endYear", "runtimeMinutes"]].astype(pd.Int64Dtype())

ValueError: invalid literal for int() with base 10: 'Reality-TV'

In [169]:
titles_ratings_df.loc[(titles_ratings_df["runtimeMinutes"].notnull()) &
                      (~titles_ratings_df["runtimeMinutes"].astype(str).str.isnumeric()),
                     "runtimeMinutes"] = pd.NA

In [170]:
titles_ratings_df[["startYear", "endYear", "runtimeMinutes"]] \
= titles_ratings_df[["startYear", "endYear", "runtimeMinutes"]].astype(pd.Int64Dtype())

In [171]:
titles_ratings_df.dtypes

tconst             object
titleType          object
primaryTitle       object
originalTitle      object
isAdult            object
startYear           Int64
endYear             Int64
runtimeMinutes      Int64
genres             object
averageRating     float64
numVotes          float64
dtype: object

In [120]:
titles_ratings_df[["startYear", "endYear"]].agg(["min","max"])

,startYear,endYear
min,1874,1906
max,2031,2030


In [82]:
titles_ratings_df["genres"] = titles_ratings_df["genres"].str.split(",")

In [177]:
titles_ratings_df[titles_ratings_df["isAdult"].isnull()]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
2994293,tt13704268,tvEpisode,Bay of the Triffids/Doctor of Doom\tBay of the...,0,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN


In [183]:
titles_ratings_df["isAdult"].fillna(0, inplace=True)

In [184]:
titles_ratings_df[titles_ratings_df["isAdult"].isnull()]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes


In [191]:
titles_ratings_df["titleType"][titles_ratings_df["isAdult"] == True].value_counts()

tvEpisode       198013
video            98192
movie             9885
tvSeries          2418
short             2022
tvMiniSeries       246
videoGame          214
tvMovie             93
tvSpecial           23
tvShort              4
Name: titleType, dtype: int64

In [96]:
titles_ratings_df[titles_ratings_df.isnull().any(axis=1)].shape[0]

9936260

In [97]:
titles_ratings_df["titleType"].unique()

array(['short', 'movie', 'tvShort', 'tvMovie', 'tvSeries', 'tvEpisode',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame', 'tvPilot'],
      dtype=object)

In [110]:
titles_ratings_df["titleType"].value_counts()

tvEpisode       7578663
short            938057
movie            649399
video            275870
tvSeries         245760
tvMovie          142199
tvMiniSeries      49112
tvSpecial         42157
videoGame         34998
tvShort            9970
tvPilot               1
Name: titleType, dtype: int64

In [245]:
movies_df = titles_ratings_df[(titles_ratings_df["titleType"] == "movie") | 
                              (titles_ratings_df["titleType"] == "tvMovie")]
movies_df.head(50)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,<NA>,45,[Romance],5.3,206.0
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,<NA>,100,"[Documentary, News, Sport]",5.3,475.0
498,tt0000502,movie,Bohemios,Bohemios,0,1905,<NA>,100,<NA>,4.1,15.0
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,<NA>,70,"[Action, Adventure, Biography]",6.0,832.0
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,<NA>,90,[Drama],4.4,20.0
610,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1907,<NA>,<NA>,[Drama],4.3,24.0
625,tt0000630,movie,Hamlet,Amleto,0,1908,<NA>,<NA>,[Drama],2.8,26.0
668,tt0000675,movie,Don Quijote,Don Quijote,0,1908,<NA>,<NA>,[Drama],4.2,20.0
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,<NA>,120,"[Adventure, Fantasy]",5.1,68.0
828,tt0000838,movie,A Cultura do Cacau,A Cultura do Cacau,0,1909,<NA>,<NA>,<NA>,NaN,NaN


In [247]:
movies_df = movies_df[movies_df["isAdult"] == False]

In [260]:
movies_df["endYear"].value_counts()

Series([], Name: endYear, dtype: Int64)

In [261]:
movies_df.drop("endYear", axis=1, inplace=True)

In [265]:
movies_df = movies_df[["tconst", "titleType", "primaryTitle", "startYear", "runtimeMinutes", "genres",
                     "averageRating", "numVotes"]]
movies_df.head()

,tconst,titleType,primaryTitle,startYear,runtimeMinutes,genres,averageRating,numVotes
8,tt0000009,movie,Miss Jerry,1894,45,[Romance],5.3,206.0
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,1897,100,"[Documentary, News, Sport]",5.3,475.0
498,tt0000502,movie,Bohemios,1905,100,<NA>,4.1,15.0
570,tt0000574,movie,The Story of the Kelly Gang,1906,70,"[Action, Adventure, Biography]",6.0,832.0
587,tt0000591,movie,The Prodigal Son,1907,90,[Drama],4.4,20.0


In [266]:
movies_df = movies_df.rename(columns={"primaryTitle": "title", "startYear": "releaseYear"})

In [267]:
movies_df.head()

,tconst,titleType,title,releaseYear,runtimeMinutes,genres,averageRating,numVotes
8,tt0000009,movie,Miss Jerry,1894,45,[Romance],5.3,206.0
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,1897,100,"[Documentary, News, Sport]",5.3,475.0
498,tt0000502,movie,Bohemios,1905,100,<NA>,4.1,15.0
570,tt0000574,movie,The Story of the Kelly Gang,1906,70,"[Action, Adventure, Biography]",6.0,832.0
587,tt0000591,movie,The Prodigal Son,1907,90,[Drama],4.4,20.0


### Remove any non-movie or non-tvMovie titles

In [195]:
for df in [titles_ratings_df, principals_df, names_df]:
    print(df.shape)

(9966186, 11)
(56893253, 6)
(12645912, 6)


### Look into principals_df to drop any principals who are not in any movies or tvMovies.
### Also only want to keep actors/actresses

In [196]:
principals_df.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,<NA>,"[""Self""]"
1,tt0000001,2,nm0005690,director,<NA>,<NA>
2,tt0000001,3,nm0374658,cinematographer,director of photography,<NA>
3,tt0000002,1,nm0721526,director,<NA>,<NA>
4,tt0000002,2,nm1335271,composer,<NA>,<NA>


In [235]:
job_types = principals_df["category"].value_counts()

In [236]:
job_types

actor                  12548490
self                    9818803
actress                 9719370
writer                  7996691
director                6572907
producer                3714286
cinematographer         1959193
composer                1918228
editor                  1904793
archive_footage          369467
production_designer      366955
archive_sound              4070
Name: category, dtype: int64

In [242]:
cast_df = principals_df[(principals_df["category"] == "actor") | (principals_df["category"] == "actress")]

In [268]:
cast_df.shape

(22267860, 6)

In [277]:
# Only keep actors/actresses that are in movies that are in the movies_df
movies_cast_df = pd.merge(cast_df, movies_df["tconst"], on="tconst")
movies_cast_df.shape

(2365581, 6)

In [279]:
movies_cast_df = movies_cast_df[["tconst", "nconst", "ordering", "characters"]]

In [280]:
movies_cast_df.head()

,tconst,nconst,ordering,characters
0,tt0000009,nm0063086,1,"[""Miss Geraldine Holbrook (Miss Jerry)""]"
1,tt0000009,nm0183823,2,"[""Mr. Hamilton""]"
2,tt0000009,nm1309758,3,"[""Chauncey Depew - the Director of the New Yor..."
3,tt0000502,nm0215752,1,<NA>
4,tt0000502,nm0252720,2,<NA>


In [525]:
# We want to split apart convert the "characters" column into a list of characters so that we can explode this column later
# First remove the "\" characters
movies_cast_df["characters"] = movies_cast_df["characters"].replace("[\[\]]", "", regex=True)

In [531]:
# Split strings into a list
movies_cast_df.loc[:, "characters"] = movies_cast_df["characters"].str.split(",")

In [536]:
# Success
movies_cast_df[movies_cast_df["characters"].apply(lambda x: isinstance(x, list) and len(x) > 1)]

,tconst,nconst,ordering,characters
21,tt0000679,nm0122665,2,"[""His Majesty the Scarecrow"", ""Para Bruin - th..."
22,tt0000679,nm0933446,3,"[""Nick Chopper - Tin Woodman"", ""The White Rabb..."
124,tt0001530,nm0738202,1,"[""Edgar Dalmore"", ""Captain Midnight""]"
135,tt0001592,nm0690702,1,"[""Ebba"", ""Sophie""]"
136,tt0001592,nm0740463,2,"[""Geheimraaden"", ""Geheimrat""]"
...,...,...,...,...
2365239,tt9909950,nm2683519,3,"[""Andy"", ""Handsome""]"
2365240,tt9909950,nm5730006,4,"[""Alya"", ""Jelita""]"
2365308,tt9910648,nm1083338,1,"[""Narrator"", ""Presenter""]"
2365431,tt9914192,nm6864841,1,"[""Paulinho Gogó"", ""Pai Paulinho Gogó""]"


In [522]:
movies_cast_df.loc[movies_cast_df["characters"].isnull(), :]

,tconst,nconst,ordering,characters
3,tt0000502,nm0215752,1,<NA>
4,tt0000502,nm0252720,2,<NA>
9,tt0000591,nm0906197,1,<NA>
10,tt0000591,nm0332182,2,<NA>
11,tt0000591,nm1323543,3,<NA>
...,...,...,...,...
2365575,tt9916706,nm5697682,3,<NA>
2365576,tt9916706,nm3272130,4,<NA>
2365577,tt9916730,nm6096005,1,<NA>
2365578,tt9916730,nm0059461,2,<NA>


In [503]:
movies_cast_df[movies_cast_df["characters"].isna()]

,tconst,nconst,ordering,characters


In [326]:
# Research more about what this is. Assuming this is ranking by amount of screen time.
# Considering keeping only top 5 or so
movies_cast_df["ordering"].value_counts()

1     544281
2     508254
3     482475
4     453012
10    107215
9     101992
8      81231
7      54438
6      27383
5       5300
Name: ordering, dtype: int64

### names_df
Only want to keep the actors/actresses that appear in a movie present in movies_df

In [281]:
names_df.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0072308,tt0053137,tt0050419"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0075213,tt0038355,tt0037382,tt0117057"
2,nm0000003,Brigitte Bardot,1934,<NA>,"actress,soundtrack,music_department","tt0057345,tt0056404,tt0054452,tt0049189"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0080455,tt0072562,tt0077975,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050976,tt0050986,tt0069467,tt0083922"


In [284]:
# Get distinct actors/actresses from movies_cast_df
dist_actors = movies_cast_df["nconst"].unique()
dist_actors

array(['nm0063086', 'nm0183823', 'nm1309758', ..., 'nm1697691',
       'nm3937824', 'nm13233318'], dtype=object)

In [285]:
# create a dataframe actors that contains only actors and actresses in dist_actors
actors_df = names_df[names_df["nconst"].isin(dist_actors)]

In [288]:
print(actors_df.shape)
actors_df.head()

(794938, 6)


,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0072308,tt0053137,tt0050419"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0075213,tt0038355,tt0037382,tt0117057"
2,nm0000003,Brigitte Bardot,1934,<NA>,"actress,soundtrack,music_department","tt0057345,tt0056404,tt0054452,tt0049189"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0080455,tt0072562,tt0077975,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050976,tt0050986,tt0069467,tt0083922"


In [294]:
# format primarProfession and knowForTitles as lists
for column in ["primaryProfession", "knownForTitles"]:
    actors_df[column] = actors_df[column].str.split(",")

C:\Users\weism\AppData\Local\Temp\ipykernel_29640\2561846182.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actors_df[column] = actors_df[column].str.split(",")


In [301]:
actors_df.loc[(actors_df["primaryProfession"].notnull()) 
              & (~actors_df["primaryProfession"].apply(lambda x: isinstance(x, list)))]

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles


In [302]:
actors_df.loc[(actors_df["knownForTitles"].notnull()) 
              & (~actors_df["knownForTitles"].apply(lambda x: isinstance(x, list)))]

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles


In [304]:
actors_df.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"[soundtrack, actor, miscellaneous]","[tt0031983, tt0072308, tt0053137, tt0050419]"
1,nm0000002,Lauren Bacall,1924,2014,"[actress, soundtrack]","[tt0075213, tt0038355, tt0037382, tt0117057]"
2,nm0000003,Brigitte Bardot,1934,<NA>,"[actress, soundtrack, music_department]","[tt0057345, tt0056404, tt0054452, tt0049189]"
3,nm0000004,John Belushi,1949,1982,"[actor, soundtrack, writer]","[tt0080455, tt0072562, tt0077975, tt0078723]"
4,nm0000005,Ingmar Bergman,1918,2007,"[writer, director, actor]","[tt0050976, tt0050986, tt0069467, tt0083922]"


In [306]:
actors_df = actors_df.rename(columns={"primaryName": "name"})

In [318]:
actors_df.shape == actors_df.loc[actors_df["birthYear"].apply(lambda x: isinstance(x, str)), :].shape

False

In [477]:
actors_df[["primaryProfession", "knownForTitles"]].astype("np.array")

TypeError: data type 'np.array' not understood

In [319]:
actors_df[["birthYear", "deathYear"]] = actors_df[["birthYear", "deathYear"]].astype('Int64')

In [323]:
# 3 core tables created in movies_df, movies_cast_df, and acros_df
for df in [movies_df, movies_cast_df, actors_df]:
    print(df.info())
    print()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 778354 entries, 8 to 9966136
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          778354 non-null  object 
 1   titleType       778354 non-null  object 
 2   title           778354 non-null  object 
 3   releaseYear     683436 non-null  Int64  
 4   runtimeMinutes  497651 non-null  Int64  
 5   genres          695030 non-null  object 
 6   averageRating   338687 non-null  float64
 7   numVotes        338687 non-null  float64
dtypes: Int64(2), float64(2), object(4)
memory usage: 54.9+ MB
None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2365581 entries, 0 to 2365580
Data columns (total 4 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   tconst      object
 1   nconst      object
 2   ordering    int64 
 3   characters  object
dtypes: int64(1), object(3)
memory usage: 90.2+ MB
None

<class 'pandas.core.frame.DataFrame'>
Int64Inde

### Splitting Apart Genres
Films can have multiple genres. This is relevant information that I want to dig in more, so I will be creating a genres table consisting of all distinct genres and a mapping table in between genres and movies.

In [409]:
movies_df.head()

,tconst,titleType,title,releaseYear,runtimeMinutes,genres,averageRating,numVotes
8,tt0000009,movie,Miss Jerry,1894,45,[Romance],5.3,206.0
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,1897,100,"[Documentary, News, Sport]",5.3,475.0
498,tt0000502,movie,Bohemios,1905,100,<NA>,4.1,15.0
570,tt0000574,movie,The Story of the Kelly Gang,1906,70,"[Action, Adventure, Biography]",6.0,832.0
587,tt0000591,movie,The Prodigal Son,1907,90,[Drama],4.4,20.0


In [344]:
# first fill in nulls with "Unspecified"
# movies_df["genres"] = movies_df["genres"].fillna("Unspecified")
# movies_df

,tconst,titleType,title,releaseYear,runtimeMinutes,genres,averageRating,numVotes
8,tt0000009,movie,Miss Jerry,1894,45,[Romance],5.3,206.0
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,1897,100,"[Documentary, News, Sport]",5.3,475.0
498,tt0000502,movie,Bohemios,1905,100,Unspecified,4.1,15.0
570,tt0000574,movie,The Story of the Kelly Gang,1906,70,"[Action, Adventure, Biography]",6.0,832.0
587,tt0000591,movie,The Prodigal Son,1907,90,[Drama],4.4,20.0
...,...,...,...,...,...,...,...,...
9966104,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,2007,100,[Documentary],NaN,NaN
9966109,tt9916692,tvMovie,Teatroteka: Czlowiek bez twarzy,2015,66,[Drama],NaN,NaN
9966116,tt9916706,movie,Dankyavar Danka,2013,<NA>,[Comedy],NaN,NaN
9966126,tt9916730,movie,6 Gunn,2017,116,Unspecified,8.3,10.0


In [410]:
# Explode movies_df and save as movies_df_xd
movies_df_xd = movies_df.explode("genres")

In [423]:
movies_df_xd.head()

,tconst,titleType,title,releaseYear,runtimeMinutes,genres,averageRating,numVotes
8,tt0000009,movie,Miss Jerry,1894,45,Romance,5.3,206.0
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,1897,100,Documentary,5.3,475.0
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,1897,100,News,5.3,475.0
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,1897,100,Sport,5.3,475.0
498,tt0000502,movie,Bohemios,1905,100,<NA>,4.1,15.0


In [424]:
# Keep only title unique id and genres. Will replace genre names with ids after creating genres table
movie_genres_df = movies_df_xd.loc[movies_df_xd["genres"].notnull(), ["tconst", "genres"]]

In [428]:
movie_genres_df["genres"].value_counts()

Drama          272213
Documentary    168060
Comedy         133726
Action          56720
Romance         52279
Thriller        48907
Crime           42926
Horror          38507
Adventure       30711
Family          23779
Biography       22119
Music           22059
Mystery         19727
History         18290
Fantasy         17568
Sci-Fi          15408
Musical         12964
Animation       11334
War             10433
Sport            8837
Western          8554
News             1776
Reality-TV       1510
Film-Noir         882
Talk-Show         411
Game-Show          94
Adult              54
Short               1
Name: genres, dtype: int64

In [429]:
movie_genres_df = movie_genres_df.reset_index(drop=True)

In [430]:
# Create genres table with distinct genres and assign an id
genres_df = pd.DataFrame(movie_genres_df["genres"].unique(), columns=["genreName"])

In [431]:
# Generate an id column using index + 1
ids_array = np.array(genres_df.index) + 1

In [432]:
ids_array

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28], dtype=int64)

In [433]:
# Create new column "genre_id"
genres_df["genre_id"] = ids_array

In [434]:
# Reorder columns
genres_df = genres_df[["genre_id", "genreName"]]

In [439]:
genres_df.head()

,genre_id,genreName
0,1,Romance
1,2,Documentary
2,3,News
3,4,Sport
4,5,Action


In [440]:
movies_df.head()

,tconst,titleType,title,releaseYear,runtimeMinutes,genres,averageRating,numVotes
8,tt0000009,movie,Miss Jerry,1894,45,[Romance],5.3,206.0
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,1897,100,"[Documentary, News, Sport]",5.3,475.0
498,tt0000502,movie,Bohemios,1905,100,<NA>,4.1,15.0
570,tt0000574,movie,The Story of the Kelly Gang,1906,70,"[Action, Adventure, Biography]",6.0,832.0
587,tt0000591,movie,The Prodigal Son,1907,90,[Drama],4.4,20.0


In [446]:
movie_genres_df.head()

,tconst,genres
0,tt0000009,Romance
1,tt0000147,Documentary
2,tt0000147,News
3,tt0000147,Sport
4,tt0000574,Action


In [447]:
# check rows before and then after. Should not lose any.
movie_genres_df.shape[0]

1039849

In [448]:
# Assign genre_id to movie_genres_df
movie_genres_df = pd.merge(movie_genres_df, genres_df, left_on="genres", right_on="genreName")

In [449]:
# Rows the same, successful id assignment.
movie_genres_df.shape[0]

1039849

In [452]:
movie_genres_df = movie_genres_df[["tconst", "genre_id"]]

In [453]:
movie_genres_df

,tconst,genre_id
0,tt0000009,1
1,tt0001175,1
2,tt0001475,1
3,tt0002026,1
4,tt0002211,1
...,...,...
1039844,tt8839532,27
1039845,tt8858536,27
1039846,tt8862888,27
1039847,tt8865024,27


In [454]:
# Keep backup of genres in movies_df. Also have movies_df_xd.
movies_df_wGenres = movies_df.copy()

In [455]:
# Now drop genres column from movies_df
movies_df.drop("genres", axis=1, inplace=True)

In [456]:
movies_df

,tconst,titleType,title,releaseYear,runtimeMinutes,averageRating,numVotes
8,tt0000009,movie,Miss Jerry,1894,45,5.3,206.0
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,1897,100,5.3,475.0
498,tt0000502,movie,Bohemios,1905,100,4.1,15.0
570,tt0000574,movie,The Story of the Kelly Gang,1906,70,6.0,832.0
587,tt0000591,movie,The Prodigal Son,1907,90,4.4,20.0
...,...,...,...,...,...,...,...
9966104,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,2007,100,NaN,NaN
9966109,tt9916692,tvMovie,Teatroteka: Czlowiek bez twarzy,2015,66,NaN,NaN
9966116,tt9916706,movie,Dankyavar Danka,2013,<NA>,NaN,NaN
9966126,tt9916730,movie,6 Gunn,2017,116,8.3,10.0


In [459]:
# Consider getting rid of the unspecified and give null value instead
pd.merge((pd.merge(movies_df, movie_genres_df, on="tconst")), genres_df, on="genre_id")["genreName"].value_counts()

Drama          272213
Documentary    168060
Comedy         133726
Action          56720
Romance         52279
Thriller        48907
Crime           42926
Horror          38507
Adventure       30711
Family          23779
Biography       22119
Music           22059
Mystery         19727
History         18290
Fantasy         17568
Sci-Fi          15408
Musical         12964
Animation       11334
War             10433
Sport            8837
Western          8554
News             1776
Reality-TV       1510
Film-Noir         882
Talk-Show         411
Game-Show          94
Adult              54
Short               1
Name: genreName, dtype: int64

In [540]:
# Now we have the following core tables
database = {"movies": movies_df, "movies_cast": movies_cast_df, "actors": actors_df, 
            "genres": genres_df, "movie_genres": movie_genres_df}

for key, df in database.items():
    print(f"**{key}**")
    print(df.info())
    print()

**movies**
<class 'pandas.core.frame.DataFrame'>
Int64Index: 778354 entries, 8 to 9966136
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          778354 non-null  object 
 1   titleType       778354 non-null  object 
 2   title           778354 non-null  object 
 3   releaseYear     683436 non-null  Int64  
 4   runtimeMinutes  497651 non-null  Int64  
 5   averageRating   338687 non-null  float64
 6   numVotes        338687 non-null  float64
dtypes: Int64(2), float64(2), object(3)
memory usage: 65.1+ MB
None

**movies_cast**
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2365581 entries, 0 to 2365580
Data columns (total 4 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   tconst      object
 1   nconst      object
 2   ordering    int64 
 3   characters  object
dtypes: int64(1), object(3)
memory usage: 154.7+ MB
None

**actors**
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7949

In [468]:
for column in movies_df.columns:
    print(movies_df[column].unique())

['tt0000009' 'tt0000147' 'tt0000502' ... 'tt9916706' 'tt9916730'
 'tt9916754']
['movie' 'tvMovie']
['Miss Jerry' 'The Corbett-Fitzsimmons Fight' 'Bohemios' ...
 'Dankyavar Danka' '6 Gunn' 'Chico Albuquerque - Revelações']
<IntegerArray>
[1894, 1897, 1905, 1906, 1907, 1908, 1909, 1910, 1912, 1911,
 ...
 2024, 2014, 2025, 2029, 2026, 2027, 2030, 1896, 2028, 2031]
Length: 138, dtype: Int64
<IntegerArray>
[   45,   100,    70,    90,  <NA>,   120,    58,    50,    51,    52,
 ...
   383, 51420,  1100,   660,   808,  2112,   590,   721,  2088,   990]
Length: 522, dtype: Int64
[ 5.3  4.1  6.   4.4  4.3  2.8  4.2  5.1  nan  4.7  4.8  3.7  3.5  3.8
  4.6  5.4  5.6  5.2  5.   4.   6.2  7.5  6.9  5.8  6.4  3.   5.5  3.6
  5.9  4.5  7.   5.7  7.6  6.6  4.9  6.3  7.2  6.5  6.7  2.6  6.8  6.1
  3.3  7.1  8.   2.7  7.3  7.4  8.4  2.2  1.4  3.1  2.   7.9  8.2  7.7
  3.2  1.   8.1  8.3  1.6  2.4  7.8  2.3  8.7  1.1  2.1  1.5  8.6  8.5
  9.2  3.9  1.8  3.4  1.3  1.9  2.5  1.7  2.9  1.2  8.8  9.3  9.4  